In [1]:
import torch
import math
import pandas as pd
import re
from tqdm import tqdm

In [2]:
file_path = 'H:\geph_package_data\data.csv'
data = pd.read_csv(file_path)
data.head()

,No.,Time,Source,Destination,Protocol,Length,Info
0,1,0.000000,127.0.0.1,127.0.0.1,TCP,76,41918 > 9809 [SYN] Seq=0 Win=65495 Len=0 MSS...
1,2,0.000006,127.0.0.1,127.0.0.1,TCP,56,"9809 > 41918 [RST, ACK] Seq=1 Ack=1 Win=0 Len=0"
2,3,0.048993,185.125.188.54,10.248.24.139,TCP,2808,"443 > 44478 [PSH, ACK] Seq=1 Ack=1 Win=443 L..."
3,4,0.048993,185.125.188.54,10.248.24.139,TCP,2808,"443 > 44478 [PSH, ACK] Seq=2741 Ack=1 Win=44..."
4,5,0.048993,185.125.188.54,10.248.24.139,TCP,2808,"443 > 44478 [PSH, ACK] Seq=5481 Ack=1 Win=44..."


In [3]:
pattern = r'(?P<Source_Port>\d+)\s*>\s*(?P<Destination_Port>\d+)\s*' \
            r'(?:\[.*\]\s*)?' \
            r'(?:Seq=(?P<Seq>\d+)\s*)?(?:Ack=(?P<Ack>\d+)\s*)?' \
            r'(?:Win=(?P<Win>\d+)\s*)?(?:Len=(?P<Len>\d+))?'
data = data[data['Info'].str.match(pattern, na=False)]

In [4]:
data.shape

(4538779, 7)

In [9]:
def parse_info_field(info):
    match = re.search(pattern, info)
    if match:
        return match.groupdict()   # Return matched groups as a dictionary
    return None  # Default if no match is found


In [10]:
parsed_info = data['Info'].apply(parse_info_field)

# Convert the parsed results into a DataFrame
parsed_df = pd.DataFrame(parsed_info.tolist())

# Concatenate the original DataFrame with the parsed data
result = pd.concat([data, parsed_df], axis=1)


In [25]:
result.dropna(inplace=True)
result.shape

(2314929, 13)

In [27]:
result.head()

,No.,Time,Source,Destination,Protocol,Length,Info,Source_Port,Destination_Port,Seq,Ack,Win,Len
1,2.0,0.000006,127.0.0.1,127.0.0.1,TCP,56.0,"9809 > 41918 [RST, ACK] Seq=1 Ack=1 Win=0 Len=0",9809,41918,1,1,0,0
6,7.0,0.049032,10.248.24.139,185.125.188.54,TCP,68.0,44478 > 443 [ACK] Seq=1 Ack=2741 Win=723 Len...,44478,443,1,5481,766,0
7,8.0,0.049062,10.248.24.139,185.125.188.54,TCP,68.0,44478 > 443 [ACK] Seq=1 Ack=5481 Win=766 Len...,44478,443,1,8221,809,0
13,14.0,0.049135,185.125.188.54,10.248.24.139,TCP,2808.0,"443 > 44478 [PSH, ACK] Seq=21921 Ack=1 Win=4...",44478,443,1,13701,809,0
19,20.0,0.049160,185.125.188.54,10.248.24.139,TCP,2808.0,"443 > 44478 [PSH, ACK] Seq=35621 Ack=1 Win=4...",44478,443,1,40330,679,0


In [33]:
specific_ip = "51.159.101.60"

# Add a new column 'Label' based on the condition
result['Label'] = (result['Source'] == specific_ip) | (result['Destination'] == specific_ip)

# Display the updated DataFrame
result.head()

,No.,Time,Source,Destination,Protocol,Length,Info,Source_Port,Destination_Port,Seq,Ack,Win,Len,Label
1,2.0,0.000006,127.0.0.1,127.0.0.1,TCP,56.0,"9809 > 41918 [RST, ACK] Seq=1 Ack=1 Win=0 Len=0",9809,41918,1,1,0,0,False
6,7.0,0.049032,10.248.24.139,185.125.188.54,TCP,68.0,44478 > 443 [ACK] Seq=1 Ack=2741 Win=723 Len...,44478,443,1,5481,766,0,False
7,8.0,0.049062,10.248.24.139,185.125.188.54,TCP,68.0,44478 > 443 [ACK] Seq=1 Ack=5481 Win=766 Len...,44478,443,1,8221,809,0,False
13,14.0,0.049135,185.125.188.54,10.248.24.139,TCP,2808.0,"443 > 44478 [PSH, ACK] Seq=21921 Ack=1 Win=4...",44478,443,1,13701,809,0,False
19,20.0,0.049160,185.125.188.54,10.248.24.139,TCP,2808.0,"443 > 44478 [PSH, ACK] Seq=35621 Ack=1 Win=4...",44478,443,1,40330,679,0,False


In [36]:
result = result.drop(columns=['Source', 'Destination', 'Info'])
result.head()

KeyError: "['Source' 'Destination' 'Info'] not found in axis"

In [37]:
result['Protocol'].value_counts()

TCP                    2314922
TCP, HiPerConTracer          7
Name: Protocol, dtype: int64

In [38]:
data_encoded = pd.get_dummies(result, columns=['Protocol'])

In [39]:
data_encoded.head()

,No.,Time,Length,Source_Port,Destination_Port,Seq,Ack,Win,Len,Label,Protocol_TCP,"Protocol_TCP, HiPerConTracer"
1,2.0,0.000006,56.0,9809,41918,1,1,0,0,False,1,0
6,7.0,0.049032,68.0,44478,443,1,5481,766,0,False,1,0
7,8.0,0.049062,68.0,44478,443,1,8221,809,0,False,1,0
13,14.0,0.049135,2808.0,44478,443,1,13701,809,0,False,1,0
19,20.0,0.049160,2808.0,44478,443,1,40330,679,0,False,1,0


In [40]:
data_encoded.to_csv('H:\geph_package_data\wireshark_encoded.csv', index=False)